In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import datetime

In [2]:
base_model = load_model("nn-final-models/base_model-1575783299.202439.kerasmodel")
doc2vec_model = load_model("nn-final-models/doc2vec_model-1575783152.668142.kerasmodel")
basicname_model = load_model("nn-final-models/basicname_model-1575829724.931042.kerasmodel")

In [3]:
def one_hot_df(df,one_hot_cols,drop_cols):
    dfret = df.copy()
    dfret = pd.get_dummies(dfret,columns=one_hot_cols).drop(columns=drop_cols)
    return dfret

# THIS RETURNS TEST DATA, NOT VALID DESPITE VARIABLE NAMES
def get_data(data_category="base", data_suffix="", one_hot_main=False, one_hot_both=False):
    train_x = pd.read_csv("data/preprocess-%s-x-train%s.csv" % (data_category, data_suffix), index_col=0)
    train_y = pd.read_csv("data/preprocess-%s-y-train%s.csv" % (data_category, data_suffix))
    valid_x = pd.read_csv("data/preprocess-%s-x-test%s.csv" % (data_category, data_suffix), index_col=0)
    valid_y = pd.read_csv("data/preprocess-%s-y-test%s.csv" % (data_category, data_suffix))
    
    if data_category == "basic_name" and data_suffix == "-correct":
        train_x.rename(inplace=True, columns={
            "launched_month_deadline_month_0": "launched_month",
            "launched_month_deadline_month_1": "deadline_month"})
        valid_x.rename(inplace=True, columns={
            "launched_month_deadline_month_0": "launched_month",
            "launched_month_deadline_month_1": "deadline_month"})
        for col in train_x.columns:
            if "Unnamed" in col:
                train_x = train_x.drop(col, axis=1)
                valid_x = valid_x.drop(col, axis=1)
        train_y["success"] = train_y["1"]
        valid_y["success"] = valid_y["1"]
    train_y.success = train_y.success.apply(lambda x: 1 if x else 0)
    valid_y.success = valid_y.success.apply(lambda x: 1 if x else 0)
    
    if one_hot_both:
        one_hot_cols = ['main_category', 'category', 'launched_month','deadline_month']
        drop_cols = []
        train_x = one_hot_df(train_x, one_hot_cols, drop_cols)
        valid_x = one_hot_df(valid_x, one_hot_cols, drop_cols)
    elif one_hot_main:
        one_hot_cols = ['main_category','launched_month','deadline_month']
        drop_cols = ['category']
        train_x = one_hot_df(train_x, one_hot_cols, drop_cols)
        valid_x = one_hot_df(valid_x, one_hot_cols, drop_cols)
    
    train_x = np.array(train_x)
    train_y = np.array(train_y.success).reshape((-1, 1))
    valid_x = np.array(valid_x)
    valid_y = np.array(valid_y.success).reshape((-1, 1))
    
    return train_x, train_y, valid_x, valid_y

data_base_one_hot_both = get_data(data_category="base", one_hot_both=True)
data_doc2vec_one_hot_both = get_data(data_category="doc2vec", one_hot_both=True)
data_basicname_one_hot_both = get_data(data_category="basic_name", data_suffix="-correct", one_hot_both=True)

In [4]:
base_predictions = base_model.predict(data_base_one_hot_both[2])
doc2vec_predictions = doc2vec_model.predict(data_doc2vec_one_hot_both[2])
basicname_predictions = basicname_model.predict(data_basicname_one_hot_both[2])

In [5]:
ts = datetime.datetime.now().timestamp()
pickle.dump(base_predictions, open("nn-final-models/base-test-predictions-%s.pickle" % ts, "wb"))
pickle.dump(doc2vec_predictions, open("nn-final-models/doc2vec-test-predictions-%s.pickle" % ts, "wb"))
pickle.dump(basicname_predictions, open("nn-final-models/basicname-test-predictions-%s.pickle" % ts, "wb"))